# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [409]:
# import libraries
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import os
from sqlalchemy import create_engine
os.chdir('/Users/tamasdinh/Dropbox/Udacity-Data-Scientist/0_Projects/4_Figure8-disaster-response')

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import re
import time
import datetime

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score

[nltk_data] Downloading package punkt to /Users/tamasdinh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tamasdinh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tamasdinh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [374]:
# load data from database
engine = create_engine('sqlite:///Fig8Messages.db')
df = pd.read_sql_table('Messages_clean', engine)

In [375]:
df.iloc[:,3:].sum(axis = 0)
# Child alone has no occurences; for modelling reasons I'll drop that variable (in a flexible, programmatic way so that if someone wnats to use the app with another dataset he/she can do so)

related                   20282
request                    4474
offer                       118
aid_related               10860
medical_help               2084
medical_products           1313
search_and_rescue           724
security                    471
military                    860
child_alone                   0
water                      1672
food                       2923
shelter                    2314
clothing                    405
money                       604
missing_people              298
refugees                    875
death                      1194
other_aid                  3446
infrastructure_related     1705
transport                  1201
buildings                  1333
electricity                 532
tools                       159
hospitals                   283
shops                       120
aid_centers                 309
other_infrastructure       1151
weather_related            7297
floods                     2155
storm                      2443
fire    

In [376]:
category_counts_series = df.loc[:,df.dtypes != 'object'].sum(axis = 0)
zero_cats = [x for x in category_counts_series[category_counts_series == 0].index]
print('The following categories have no records to them:\n {}'.format(zero_cats))
print('-------- Removing {} category label(s) ---------'.format(len(zero_cats)))

The following categories have no records to them:
 ['child_alone']
-------- Removing 1 category label(s) ---------


In [377]:
df.drop(zero_cats, axis = 1, inplace = True)
X = df[['message']].values.ravel()
y = df.drop(['message', 'genre', 'original'], axis = 1).values

### 2. Write a tokenization function to process your text data

In [378]:
def tokenize(text):
    clean_text = re.sub(r'[^a-zA-Z]', ' ', text)
    clean_text = re.sub(r'\b[a-zA-Z]{1,2}\b', '', clean_text)
    clean_text = word_tokenize(clean_text)
    clean_text = [WordNetLemmatizer().lemmatize(word.lower().strip(' ().,-:![]``#''@? ')) for word in clean_text if word not in stopwords.words('english')]
    clean_text = [word for word in clean_text if word not in ['', '@', '?', '-']]
    return clean_text

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [388]:
pipeline = \
Pipeline([
    ('countvect', CountVectorizer(tokenizer = tokenize)),
    ('tf-idf', TfidfTransformer()),
    ('multi-output-classification', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [389]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [390]:
print('Fitting pipeline model on training data\n------------------------------')
start = time.time()
model = pipeline
model.fit(X_train, y_train)
end = time.time()
print('\nElapsed time: {:.0f} seconds'.format(end-start))
print('\nGenerating predictions on test data\n------------------------------')
start = time.time()
y_preds = model.predict(X_test)
end = time.time()
print('Elapsed time: {:.0f} seconds'.format(end-start))

Fitting pipeline model on training data
------------------------------

Elapsed time: 158 seconds

Generating predictions on test data
------------------------------
Elapsed time: 35 seconds


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [391]:
y_test.sum(axis = 0)

array([5062, 1073,   29, 2739,  534,  319,  177,  120,  200,  404,  700,
        596,   96,  159,   74,  252,  295,  873,  437,  322,  320,  151,
         28,   79,   21,   80,  301, 1854,  540,  604,   54,  630,  148,
        347, 1231])

In [392]:
y_preds.sum(axis = 0)

array([5471,  611,    0, 2287,   64,   52,    9,    4,   50,  138,  493,
        213,   18,    8,    4,   23,   80,   96,   14,   48,   62,    8,
          0,    0,    0,    0,    7, 1343,  232,  356,    4,  512,   16,
         50,  564])

In [393]:
print('Generating classification metrics\n------------------------------')
start = time.time()
for i in range(y_preds.shape[1]):
    print(category_counts_series.index[i], '\n-------------------------------')
    print(classification_report(y_test[:, i], y_preds[:, i]))
end = time.time()
print('Elapsed time: {:.0f} seconds'.format(end-start))

Generating classification metrics
------------------------------
related 
-------------------------------
             precision    recall  f1-score   support

          0       0.61      0.46      0.53      1533
          1       0.85      0.90      0.87      4980
          2       0.23      0.46      0.31        41

avg / total       0.79      0.80      0.79      6554

request 
-------------------------------
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5481
          1       0.78      0.45      0.57      1073

avg / total       0.88      0.89      0.88      6554

offer 
-------------------------------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6554

aid_related 
-------------------------------
             precision    recall  f1-score   support

          0      

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [397]:
pipeline = \
Pipeline([
    ('countvect', CountVectorizer(tokenizer = tokenize)),
    ('tf-idf', TfidfTransformer()),
    ('multi-output-classification', MultiOutputClassifier(RandomForestClassifier(n_jobs = -1)))
])

In [449]:
parameters = {
    'countvect__max_df': [1.0],
    'countvect__max_features': [20000],
    'multi-output-classification__estimator__max_depth': [40],
    'multi-output-classification__estimator__n_estimators': [100],
    'multi-output-classification__estimator__min_samples_split': [5]
}

cv = GridSearchCV(pipeline, param_grid = parameters, n_jobs = -1, verbose = 2)

In [450]:
start = time.time()
print('Starting grid search optimization at {} -------'.format(datetime.datetime.now().time()))
cv.fit(X_train, y_train)
end = time.time()
print('Total time elapsed: {:0f} minutes ------'.format((end-start)/60))

Starting grid search optimization at 03:31:19.734484 -------
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] countvect__max_df=1.0, countvect__max_features=20000, multi-output-classification__estimator__max_depth=40, multi-output-classification__estimator__min_samples_split=5, multi-output-classification__estimator__n_estimators=100 
[CV] countvect__max_df=1.0, countvect__max_features=20000, multi-output-classification__estimator__max_depth=40, multi-output-classification__estimator__min_samples_split=5, multi-output-classification__estimator__n_estimators=100 
[CV] countvect__max_df=1.0, countvect__max_features=20000, multi-output-classification__estimator__max_depth=40, multi-output-classification__estimator__min_samples_split=5, multi-output-classification__estimator__n_estimators=100 
[CV]  countvect__max_df=1.0, countvect__max_features=20000, multi-output-classification__estimator__max_depth=40, multi-output-classification__estimator__min_samples_split=5, multi-out

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  6.0min finished


Total time elapsed: 8.333241 minutes ------


In [451]:
y_preds = cv.best_estimator_.predict(X_test)

In [452]:
print('Generating classification metrics\n------------------------------')
start = time.time()
for i in range(y_preds.shape[1]):
    print(category_counts_series.index[i], '-----------')
    print(classification_report(y_test[:, i], y_preds[:, i]))
end = time.time()
print('Elapsed time: {:.0f} seconds'.format(end-start))

Generating classification metrics
------------------------------
related -----------
             precision    recall  f1-score   support

          0       0.87      0.05      0.09      1533
          1       0.77      1.00      0.87      4980
          2       0.00      0.00      0.00        41

avg / total       0.79      0.77      0.68      6554

request -----------
             precision    recall  f1-score   support

          0       0.86      1.00      0.92      5481
          1       0.93      0.16      0.27      1073

avg / total       0.87      0.86      0.82      6554

offer -----------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6554

aid_related -----------
             precision    recall  f1-score   support

          0       0.72      0.93      0.81      3815
          1       0.83      0.49      0.61      

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [425]:
cv.best_params_

{'countvect__max_df': 0.75,
 'countvect__max_features': 10000,
 'multi-output-classification__estimator__max_depth': 5,
 'multi-output-classification__estimator__min_samples_split': 20,
 'multi-output-classification__estimator__n_estimators': 200}

In [432]:
cv.grid_scores_

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:762: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.19582, std: 0.00454, params: {'countvect__max_df': 0.75, 'countvect__max_features': 5000, 'multi-output-classification__estimator__max_depth': 5, 'multi-output-classification__estimator__min_samples_split': 10, 'multi-output-classification__estimator__n_estimators': 200},
 mean: 0.19587, std: 0.00417, params: {'countvect__max_df': 0.75, 'countvect__max_features': 5000, 'multi-output-classification__estimator__max_depth': 5, 'multi-output-classification__estimator__min_samples_split': 20, 'multi-output-classification__estimator__n_estimators': 200},
 mean: 0.19562, std: 0.00435, params: {'countvect__max_df': 0.75, 'countvect__max_features': 5000, 'multi-output-classification__estimator__max_depth': 5, 'multi-output-classification__estimator__min_samples_split': 30, 'multi-output-classification__estimator__n_estimators': 200},
 mean: 0.19129, std: 0.00295, params: {'countvect__max_df': 0.75, 'countvect__max_features': 5000, 'multi-output-classification__estimator__max_depth': 10

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.